In [1]:
texto = """Al igual que la Computación comienza su andadura como respuesta a la pregunta sobre las posibilidades de
automatización de los procedimientos matemáticos, la IA surge de forma natural al plantear hasta qué punto aquellos
procesos que consideramos propios de una inteligencia natural (en particular, la humana) podrían ser automatizables
en mecanismos artificiales, tengan éstos un componente físico concreto o sólo conceptual."""

print(len(texto))

426


In [2]:
vocabulario = sorted(list(set(texto)))
print(vocabulario)

['\n', ' ', '(', ')', ',', '.', 'A', 'C', 'I', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'z', 'á', 'é', 'í', 'ó']


In [3]:
print(f"Vocabulario: {''.join(vocabulario)} tamaño: {len(vocabulario)}")

Vocabulario: 
 (),.ACIabcdefghilmnopqrstuzáéíó tamaño: 33


In [4]:
stoi = {ch:i for i,ch in enumerate(vocabulario)}
print(stoi)
itos = {i:ch for i,ch in enumerate(vocabulario)}
print(itos)

{'\n': 0, ' ': 1, '(': 2, ')': 3, ',': 4, '.': 5, 'A': 6, 'C': 7, 'I': 8, 'a': 9, 'b': 10, 'c': 11, 'd': 12, 'e': 13, 'f': 14, 'g': 15, 'h': 16, 'i': 17, 'l': 18, 'm': 19, 'n': 20, 'o': 21, 'p': 22, 'q': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'z': 28, 'á': 29, 'é': 30, 'í': 31, 'ó': 32}
{0: '\n', 1: ' ', 2: '(', 3: ')', 4: ',', 5: '.', 6: 'A', 7: 'C', 8: 'I', 9: 'a', 10: 'b', 11: 'c', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'l', 19: 'm', 20: 'n', 21: 'o', 22: 'p', 23: 'q', 24: 'r', 25: 's', 26: 't', 27: 'u', 28: 'z', 29: 'á', 30: 'é', 31: 'í', 32: 'ó'}


In [7]:
def tokeniza(s):
    ret = [stoi[c] for c in s]
    return ret
def destokeniza(ltokens):
    ret = "".join([itos[i] for i in ltokens])
    return ret

In [9]:
print(tokeniza("pepito"))
print(destokeniza([22, 13, 22, 17, 26, 21]))

[22, 13, 22, 17, 26, 21]
pepito


In [12]:
from nltk.tokenize import WhitespaceTokenizer
tk = WhitespaceTokenizer()
texto = "¿Cuánto tiempo pasó desde que comí una manzana?"
texto_tokenizado = tk.tokenize(texto)
print(texto_tokenizado)

['¿Cuánto', 'tiempo', 'pasó', 'desde', 'que', 'comí', 'una', 'manzana?']


In [13]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
texto = "¿Cuánto tiempo pasó desde que comí una manzana?"
texto_tokenizado1 = WordPunctTokenizer().tokenize(texto)
texto_tokenizado2 = TreebankWordTokenizer().tokenize(texto)
print(texto_tokenizado1)
print(texto_tokenizado2)

['¿', 'Cuánto', 'tiempo', 'pasó', 'desde', 'que', 'comí', 'una', 'manzana', '?']
['¿Cuánto', 'tiempo', 'pasó', 'desde', 'que', 'comí', 'una', 'manzana', '?']


In [14]:
import stanza
stanza.download("es")
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')
texto_pablito = "Pablito clavó un clavito cuantos clavitos clava pablito"
doc = nlp(texto_pablito)
print(*[f'Palabra: {word.text+" "}\tLemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

ModuleNotFoundError: No module named 'stanza'

## Token bueno

In [18]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [19]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [20]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [30]:
files = [f"wikitext-103-raw-v1/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [32]:
tokenizer.save("tokenizer-wiki.json")

In [33]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [34]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")

In [35]:
print(output.tokens)

['Hello', ',', 'y', "'", 'all', '!', 'How', 'are', 'you', '[UNK]', '?']


In [36]:
print(output.ids)

[27253, 16, 93, 11, 5097, 5, 7961, 5112, 6218, 0, 35]


In [38]:
print(output.offsets[9])

(26, 27)


In [39]:
sentence = "Hello, y'all! How are you 😁 ?"
sentence[26:27]

'😁'

In [40]:
tokenizer.token_to_id("[SEP]")

2

In [41]:
from tokenizers.processors import TemplateProcessing
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)